In [1]:
import numpy as np
from utils import *
import matplotlib.pyplot as plt

In [2]:
f = open("./dataset/MITD/video_title.txt")
titlemix = f.read()

titlelist = []
for line in titlemix.splitlines():
    if "mp4" in line:
        titlelist.append(line)

In [3]:
# def rgb2gray(rgb):
#     return  rgb[...,:3] @ [0.2989, 0.5870, 0.1140]

In [4]:
path = "./dataset/MITD/"

originals = []
interpolated = []
stats = []

for i in range(len(titlelist)):
    file_in = path+"videos/erupting_val/"+titlelist[i]
    file_out_interpolated = path+"videos_for_poster/eruption_test_{:02d}_interpolated.mp4".format(i)
    
    data_original, _ = video_to_frames(file_in)
    data_original =  data_original[:16]
    imgs = data_original[::15]
    data_interpolated = generate_resized_and_interpolated_video(data_original, imgs, True)
    
    imageio.mimwrite(file_out_interpolated, data_interpolated, fps=10)
    originals.append(data_original)
    interpolated.append(data_interpolated)
    
    stats.append(L2Difference(data_original, data_interpolated))
print("avg interpolated L2 dist = {} ".format(np.mean(stats)))
print("SD of interpolated L2 dist = {} ".format(np.std(stats)))


avg interpolated L2 dist = 3222.5635602144794 
SD of interpolated L2 dist = 2959.341553517907 


In [5]:
# for video_id in range(4):
#     fig = plt.figure(figsize=(10 * 2, 2))
#     for frame_id in range(0, 16):
#         ax = fig.add_axes([(frame_id+1) * 1 / 15., 0, (frame_id + 2) * 1 / 15., 1],
#                           xmargin=0, ymargin=0)
#         ax.imshow(originals[video_id][frame_id]/255)
#         ax.axis('off')

In [6]:
# for video_id in range(4):
#     fig = plt.figure(figsize=(10 * 2, 2))
#     for frame_id in range(0, 16):
#         ax = fig.add_axes([(frame_id+1) * 1 / 15., 0, (frame_id + 2) * 1 / 15., 1],
#                           xmargin=0, ymargin=0)
#         ax.imshow(interpolated[video_id][frame_id]/255)
#         ax.axis('off')

In [7]:
mean fvd is 955.29

SyntaxError: invalid syntax (<ipython-input-7-6843986d37ae>, line 1)

In [35]:
np.array(originals).shape

(100, 16, 64, 64, 3)

### video inbetweening

In [36]:
from os import listdir
from os.path import isfile, join
mypath = '/home/xih108/Video_Completion/dataset/MITD/eruption/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [37]:
import pickle

file = open(mypath + 'preds.p', 'rb')
preds = pickle.load(file)


In [43]:
path = "/home/xih108/Video_Completion/dataset/MITD/"

stats_gen = []
stats_inter = []
generate = []
for i in range(100):
    generate += [np.repeat(preds[i],3).reshape((16,64,64,3))]
    stats_gen += [L2Difference(originals[i], generate[i])]
    stats_inter += [L2Difference(originals[i], interpolated[i])]
    for frame in range(16):
        imageio.imwrite(path+'erupt_original/' +str(i) + "_"+str(frame)+".png",originals[i][frame])
        imageio.imwrite(path+'erupt_interpolate/'+str(i) +"_"+str(frame)+".png",interpolated[i][frame])
        imageio.imwrite(path+'erupt_generate/' +str(i) +"_"+ str(frame)+".png",generate[i][frame])

In [42]:
ls dataset/MITD/

erupt_generate     erupt_original/    skiing/           videos_for_poster/
erupt_interpolate  extracted_images/  skiing_title.txt  video_title.txt
eruption/          extract.py         videos/


In [11]:
print(np.mean(stats_gen))
print(np.std(stats_gen))

35649.6770328735
6988.639288288661


In [12]:
print(np.mean(stats_inter))
print(np.std(stats_inter))

3222.5635602144794
2959.341553517907


In [18]:
# coding=utf-8
# Copyright 2020 The Google Research Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Example code that computes FVD for some empty frames.
The FVD for this setup should be around 131.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow.compat.v1 as tf
import frechet_video_distance as fvd

# Number of videos must be divisible by 16.
NUMBER_OF_VIDEOS = len(originals)
VIDEO_LENGTH = 15
batch_size = 16

def loop(idx):

      with tf.Graph().as_default():
    
        video_set1 = tf.convert_to_tensor(np.array(originals[idx*batch_size:(idx+1)*batch_size]), dtype=tf.float32)
        video_set2 = tf.convert_to_tensor(np.array(interpolated[idx*batch_size:(idx+1)*batch_size]), dtype=tf.float32)
        video_set3 = tf.convert_to_tensor(np.array(generate[idx*batch_size:(idx+1)*batch_size]), dtype=tf.float32)
  
        first_set_of_videos = video_set1
        second_set_of_videos = video_set2
        third_set_of_videos = video_set3


        result_inter = fvd.calculate_fvd(
            fvd.create_id3_embedding(fvd.preprocess(first_set_of_videos,
                                                    (224, 224))),
            fvd.create_id3_embedding(fvd.preprocess(second_set_of_videos,
                                                    (224, 224))))
        result_gen = fvd.calculate_fvd(
            fvd.create_id3_embedding(fvd.preprocess(first_set_of_videos,
                                                    (224, 224))),
            fvd.create_id3_embedding(fvd.preprocess(third_set_of_videos,
                                                    (224, 224))))

        with tf.Session() as sess:
              sess.run(tf.global_variables_initializer())
              sess.run(tf.tables_initializer())
              print("FVD is: %.2f." % sess.run(result_inter))
              print("FVD is: %.2f." % sess.run(result_gen))
              

In [19]:
loop(0)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


FVD is: 982.80.
FVD is: 1958.04.


In [20]:
loop(1)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


FVD is: 441.28.
FVD is: 1918.82.


In [21]:
loop(2)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


FVD is: 646.31.
FVD is: 2051.93.


In [22]:
loop(3)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


FVD is: 689.45.
FVD is: 1693.83.


In [23]:
loop(4)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


FVD is: 632.63.
FVD is: 2132.18.


In [24]:
loop(5)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


FVD is: 605.74.
FVD is: 1908.33.


In [28]:
3998.21 / 6

666.3683333333333

In [29]:
11663.13 / 6

1943.8549999999998